# Projet info classes

In [183]:
# imports
import numpy as np
from uuid import uuid4 # pour identifiants uniques générés automatiquement
import pygraphviz as pgv
from PIL import Image

In [184]:
class Node:
    def __init__(self, lat : float, long : float):
        self.id = uuid4()
        self.lat = lat
        self.long = long
        self.children = []  # dico où les cles sont les identifiants, les valeurs [id,  et les valeurs le poids des aretes
    
    def dist(node1, node2):
        return np.sqrt((node1.lat - node2.lat)**2 +(node1.long - node2.long)**2)
    
    def new_child(self, node):  # crée un fils 
        # distance au père est la distance euclidienne
        self.children.append(node)
    
    def __eq__(self, node):
        if self.id == node.id:
            return True
        else:
            return False
    

        

In [185]:
class Client(Node): 
    def __init__(self, lat : float, long : float):
        self.id = uuid4()
        self.lat = lat
        self.long = long
        self.children = []
        self.str = "client"
        # self.colis = colis # nombre de colis désirés
# probablement inutile ... garder pour représentation graphique??
def make_client(lat : float, long : float):
    new_client = Client(lat, long)
    return new_client
    
    
class Colis(Node):
    def __init__(self, size, entrepot: Node, destination):
        self.id = uuid4()
        self.size = size
        self.entrepot = entrepot
        self.client = make_client(destination[0], destination[1])
        self.children = []
        

class Garage(Node):
    def __init__(self, lat: float, long: float, nb_camions: int, nb_legers: int):
        self.id = uuid4()
        self.lat = lat
        self.long = long
        self.nb_camions = nb_camions
        self.nb_legers = nb_legers
        self.children = []
        self.str = "garage"

class Entrepot(Node):
    def __init__(self, lat : float, long : float, max_camions : int, max_legers : int, capacite : int):
        self.id = uuid4() # retrouver comment faire des identifiants uniques (avec un itérable?)
        self.lat = lat
        self.long = long
        self.children = []
        self.max_camions = max_camions
        self.max_legers = max_legers
        self.capacite = capacite
        self.str = "entrepot"

class Route: # j'aurais tendance à mettre ces infos (enfin etroite uniquement) dans le dico directement
    def __init__(self, vitesse, etroite : bool, dist):
        self.vitesse = vitesse # vitesse max autorisée en km/h
        self.etroite = etroite
        self.dist = dist

In [186]:
class Vehicule:
    def __init__(self, capacite, pollution, dist_max, temps, route_etroite : bool):
        self.capacite = capacite
        self.pollution = pollution
        self.dist_max = dist_max
        self.route_etroite = route_etroite
        self.temps = temps # temps du camioneur 
class Camion(Vehicule):
    def __init__(self, capacite, pollution, dist_max, route_etroite = False):
        self.capacite = capacite
        self.pollution = pollution
        self.dist_max = dist_max
        self.route_etroite = route_etroite
        
class Leger(Vehicule):
    def __init__(self, capacite, pollution = 0, dist_max = 10, route_etroite = True):
        self.capacite = capacite
        self.pollution = pollution
        self.dist_max = dist_max
        self.route_etroite = route_etroite    

In [226]:
class Graph:
    def __init__(self, garage, entrepots: [Entrepot], points_relais: [Entrepot], colis: [Colis]):
        self.garage = garage #la racine 
        self.entrepots = entrepots # liste des entrepots (fixée)
        self.points_relais = points_relais
        self.colis = colis # liste des colis à livrer le jour n

    def make_graph(self):
        # self.graph_list.append(self.garage)
        for e in self.entrepots:
            self.garage.new_child(e) # arete orientée du garage vers l'entrepot
            colis_e = [] # liste des paquets qui partent de e
            
            for p in self.colis:
                if p.entrepot == e:
                    colis_e.append(p)
            for p in colis_e:
                e.new_child(p.client) # arete orientée de l'entrepot vers le client
                for pp in colis_e:
                    if pp.id != p.id:
                        p.new_child(pp.client) # NB: pour le moment on "perd" le paquet dans la construction du graphe
        
        for r in self.points_relais:
            for p in self.colis:
                if dist(r, p) < 5: # périmètre de 5 km
                    r.new_child(p.client)
                    p.client.new_child(r)  
                    
                    
    def __repr__(self): # a retravailler
        node = self.garage
        rep = f"{node.str}: {self.garage.lat, self.garage.long} \n | \n"
        file = []
        file.append(node)
        while len(file)>0:
            print(file)
            node = file[0]
            children = node.children
            ch = ""
            for c in children[0: len(children)-1]:
                ch += c.str + ": " + f"{c.lat, c.long}" + " - "
                file.append(c)
            if len(children) > 0:
                file.append(children[len(children)-1])
                ch += children[len(children)-1].str + ": " + f"{children[len(children)-1].lat, children[len(children)-1].long}" + "\n |"
            rep += ch
            file.pop(0)
            return rep

In [230]:
# test
g = Garage(150, 50, 40, 60)
e1 = Entrepot(100, 100, 10, 15, 5000)
e2 = Entrepot(200, 100, 20, 10, 4000)
p1 = Colis(20, e1, [104, 120])
entrepots = [e1, e2]
points_relais = []
paquets = [p1]
G = Graph(g, entrepots, points_relais, paquets)
G.make_graph()
a = G.garage.children
for e in a:
    print(e.children)


[]


In [ ]:
def trace_graph(graph):
    G = pgv.AGraph()
    root = graph.garage
    G.add_node(root)
    
    
    
    G.layout()
    G.layout(prog='dot')
    G.draw('file.png') 
    a = Image.open('file.png')
    a.show()

In [221]:
G = pgv.AGraph()

In [60]:
G.add_node(g)

In [61]:
G.write("file.dot")
